<a href="https://colab.research.google.com/github/pelanat1207/Project-1/blob/master/Project_1__MAAB_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://img.icons8.com/ios/50/000000/cottage--v1.png"/>

#**MAAB-Israel: Management Application of  Apartments' Buildings in Israel** 


---

**Anat Peled \ July 2020 \ NAYA College**


In [ ]:
import re
from collections import defaultdict
import itertools
from datetime import datetime, timedelta              
from decimal import Decimal
import smtplib, ssl
import getpass as gp
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

## **Background**

This application was made in order to ease the managment of an apartments building in Israel. The main user of this application should be the building committee or a proffesional buildings' managment company. The main purpose of this application is to keep transparency of the expenses and the building's account stauts at all time to all apartmens' owners.

[github link](https://github.com/pelanat1207/Project-1)

##**Components**
The application is made of three classes (Owner, Apartment, Building). Classes are associated to each other as explained later. Reports are available both as screen output and e-mail. 


#**E-mailing with Python**
---

In order to send the reports by e-mail, I established a new gmail account: MAABIsrael1@gmail.com. I used the smtplib, ssl, email.MIME and getpass for the purpose of connection establishment and secured password input.

There are 3 functions that enable the reports mailing:
1. **send_report_by_mail():** Organize the e-mail structure both in plain text and HTML and sends a secure e-mail
2. **convert_to_comma_file():** Converts a plain text table to a comma text file
3. **create_HTML_table():** Converts a text file to an HTML table

The **send_HTML_report()**, the 4th function, combines all of the above functions. This is the function to be used on a report.


For further reading see:
* [Real Python - Sending Emails With Python](https://realpython.com/python-send-email/#option-1-setting-up-a-gmail-account-for-development).
* [VIDIGEST: Generating An HTML Table Using PYTHON 3](https://vidigest.com/2018/12/02/generating-an-html-table-from-file-data-using-python-3/)

In [ ]:
def send_report_by_mail (report_name, receiver_email, receiver_name, content):

    sender_email = "MAABisrael1@gmail.com"
    message = MIMEMultipart("alternative")
    message["Subject"] = report_name
    message["From"] = sender_email
    message["To"] = receiver_email
    password = gp.getpass("Type your password and press enter: ")

    # Create the plain-text and HTML version of your message
    text = f'{content}'
    html = """\
    <html> 
    <p> Dear {0}, </p> 
    <head> <style> 
          table, th, td {{ border: 1px solid black; border-collapse: collapse; }}
          th, td {{ padding: 5px; }}
    </style> </head> 
    <body> <p> {1} </p> </body>
    </html>
    """.format(receiver_name, content)

    # Turn these into plain/html MIMEText objects
    part1 = MIMEText(text, "plain")
    part2 = MIMEText(html, "html")

    # Add HTML/plain-text parts to MIMEMultipart message
    # The email client will try to render the last part first
    message.attach(part1)
    message.attach(part2)

    # Create secure connection with server and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(
            sender_email, receiver_email, message.as_string()
        )

# Since ASCII plain text tables are not displayed well in HTML, we must convert it to a comma file
# which in turm will be converted to HTML table. 

def convert_to_comma_file(text):
  # This methos takes a multiline string and convert it to a myfile.txt file wilth lines seperated by commas:
    text_file = open("myfile.txt", "w")
    my_str = ''
    for line in text.split('\n'):
      a = line.strip()
      if len(a) == a.count('-'):
          a = a.replace('-','')
      else:
          a = a.replace('|',',').strip()
      if len(a) != 0:
          my_str = my_str + a + '\n'     
    text_file.write(my_str)
    text_file.close()

def create_HTML_table(input_file_name):
    # This function is taken from: https://vidigest.com/2018/12/02/generating-an-html-table-from-file-data-using-python-3/
    # The script requires 1 argument: the input file name.
    # It expects a comma-separated input file to parse into an html table,
    # and assumes that the column headers are located in the first row.

    filein = open(input_file_name, "r")
    fileout = open("html_table.html", "w")
    data = filein.readlines()
    table = "<table>\n"

    # Create the table's column headers
    header = data[0].split(",")
    table += "  <tr>\n"
    for column in header:
        table += "    <th> {0} </th>\n".format(column.strip())
    table += "  </tr>\n"

    # Create the table's row data
    for line in data[1:]:
        row = line.split(",")
        table += "  <tr>\n"
        for column in row:
            table += "    <td> {0} </td>\n".format(column.strip())
        table += "  </tr>\n"
    table += "</table>"
    fileout.writelines(table)
    fileout.close()
    filein.close

def send_HTML_report(func, receiver_email, receiver_name, report_name):
  convert_to_comma_file(func)
  create_HTML_table("myfile.txt")
  with open('html_table.html', 'r') as a:
    file = a.read()
    send_report_by_mail (report_name, receiver_email, receiver_name, file)

#**Class Owner**
----

This class contains the data of the aparment's owners. It maintains the contact information of the owners: ID, name, mail & phone number.
The class functions are: __ new __ (), __ init __ (), __ str__(), remove_owner(), update_phone(), update_email() and valid_mail().
It has one stand-alone report for all the documented owners in the class.

##**Instantiation & Basic Functions**



In [ ]:
class Owner():

  owner_dict = defaultdict(list)

  def __new__ (cls, id, name, phone, mail):
     """
     The __new__ method is similar to the __init__ method, but if both exist, __new__ method executes first.
     the __new__ method checks if ID already exist in owners list and prevent duplicates.
     """
     if id not in Owner.owner_dict.keys():
         return object.__new__(cls)
     else:
         print(f'ID {id} already exists!')
         return None

  def __init__ (self, id, name, phone, mail):
      self.id = str(id)
      self.name = str(name)
      self.phone = str(phone)
      if Owner.valid_mail(self, mail):
         self.mail = str(mail)
      else:
         self.mail = ''
         print("Invalid email address. e-mail set to blank!")
      Owner.owner_dict[id] = [self.name, self.phone, self.mail]
         
  def __str__ (self):
      try:
        owners_string = ''
        header = '{:10}|{:15}|{:^12}|{:20}\n'.format('ID','Name','Phone','e-mail')
        line = '-'*len(header)+'\n'
        person_line = '{:10}|{:15}|{:^12}|{:20}\n'.format(self.id, self.name, self.phone, self.mail)
        return (header+line+person_line)
      except:
        return 'No Occurance'
  
  def remove_owner (self):  
      Owner.owner_dict.pop(self.id, 'No Occurance')
      self.id = None
      self.name = None
      self.phone = None
      self.mail = None

  def update_phone(self, new_phone):
      self.phone = new_phone
  
  def valid_mail(self, mail_address):
      pattern = re.compile(r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$")
      valid = pattern.match(mail_address)
      return valid

  def update_email(self, new_email):
      if Owner.valid_mail(self, new_email):
          self.mail = new_email
          Owner.owner_dict[self.id][2] = new_email
      else:
          self.mail = ''
          print("Email update failed due to invalid email address. e-mail set to blank!")

##**Demonstration**

In [ ]:
# Creating 3 instances (owner3 is expected to fail)
owner1 = Owner('067983451','Anat Peled', '0543332345', 'anat@gmail.com')
owner2 = Owner('012345678', 'Carmit Assael', '0506236751', 'cmassael@gmail.com')
owner3 = Owner('012345678', 'Anonimus', '3333333', 'NoWay@gmail.co.il')

ID 012345678 already exists!


In [ ]:
# Print an instance and number of instances in class
print(owner1)
print('Owners list contains', len(Owner.owner_dict), 'owners')

ID        |Name           |   Phone    |e-mail              
-------------------------------------------------------------
067983451 |Anat Peled     | 0543332345 |anat@gmail.com      

Owners list contains 2 owners


In [ ]:
# Creating a third instance
owner3 = Owner('012345679', 'Anonimus', '3333333', 'NoWay@gmail.co.il')

In [ ]:
# Update email to owner1 and print it
owner1.update_email('pelanat1207@gmail.com')
print(owner1)

ID        |Name           |   Phone    |e-mail              
-------------------------------------------------------------
067983451 |Anat Peled     | 0543332345 |pelanat1207@gmail.com



In [ ]:
# remove an instance from the list and verify it was removed
owner3.remove_owner()
print(owner3)
print('Owners list contains', len(Owner.owner_dict), 'owners')

No Occurance
Owners list contains 2 owners


##**Report 1: Owners Contact List**

In [ ]:
def PrintContactList():
  #Report 1: A table of all owners (even if no aprtment is linked)
  owners_string = ''
  header = '{:10}|{:15}|{:^12}|{:20}\n'.format('ID','Name','Phone','e-mail')
  line = '-'*len(header)+'\n'
  for owner in Owner.owner_dict.items():
          person_line = '{:10}|{:15}|{:^12}|{:20}\n'.format(owner[0], owner[1][0], owner[1][1], owner[1][2])
          owners_string += person_line
  return header + line + owners_string

In [ ]:
print(PrintContactList())
send_HTML_report(PrintContactList(),'pelanat1207@gmail.com','Anat','Owners List')

ID        |Name           |   Phone    |e-mail              
-------------------------------------------------------------
067983451 |Anat Peled     | 0543332345 |pelanat1207@gmail.com
012345678 |Carmit Assael  | 0506236751 |cmassael@gmail.com  

Type your password and press enter: ··········


#**Class Apartment**

This class associates to **both** the Building & Owner classes. Association has 2 forms:
> *Aggregation*: this represents a *'has-a'* relationship. In both, the entries can survive individually which means ending one Object will not effect the other Object. Therefore we can say that the owner and apartment class are aggregated, since each of them can exists without the other.

> *Composition*: this is a restricted form of aggregation in which two classes are highly dependent on each other. We can say a building is composed of one or more apartments and therefore the classes are composed of each other. 

The instances of this class are formed **during** the instantination of a building. 

The default attributes of the Apartment class are: apartment number, ratio, monthly fee and owner. Each instance has its own payments dictionary to record all payments per year.

The functions of this class are: __init__ (), __str__ (), add_owner(), del_owner(), init_payments_dict(), apartment_yearly_balance() and get_payment().

##**Instantiation  & Basic Functions**

> *Tip: This code must be executed but **not** tested prior to the Building class*

In [ ]:
class Apartment():

  def __init__(self, build, num):
    self.num = f'{build:^6}/{num:>2}'
    self.owner = ''
    self.owner_name = ''
    self.ratio = float(1/ Building.build_db[build][3]) # = app_num (number of apartments)
    self.a_monthly_fee = round(float(Building.build_db[build][4] * self.ratio))
    self.a_balance = round(self.a_monthly_fee * (12 - datetime.now().month))
    self.payments_dict = defaultdict(list)
    self.init_payments_dict(datetime.now().year)

  def __str__(self):
    apartment_string = ''
    header = '{:9}|{:^6}|{:^8}|{:15}|{:^9}\n'.format('# Apart', 'Ratio', 'M. Fee', 'Owner Name', 'Balance')
    line = '-'*len(header)+'\n'
    apartment_line = '{:^9}|{:^6.1%}|{:^8}|{:15}|{:^9}\n'.format(self.num, self.ratio, self.a_monthly_fee, self.owner_name, self.a_balance)
    return (header+line+apartment_line)
  
  def add_owner(self, owner):
    # Match an owner from owners list to an apartment (only ID and name)
    self.owner = owner
    self.owner_name = Owner.owner_dict[owner][0]
  
  def del_owner(self):
    # delets the owner from the apartment attribute, not from the owners list!
    self.owner = None
    self.owner_name = None
  
  def init_payments_dict(self, year):
    self.payments_dict[year] = [self.ratio, self.a_monthly_fee, self.a_balance, 0]

  def apartment_yearly_balance(self, year):
    self.a_balance = round(self.a_monthly_fee * 12 - self.payments_dict[year][3])
    self.payments_dict[year][2] = self.a_balance

  
  def get_payment(self, year):
    global payments_dict
    payment = int(input("Enter the sum of payment recieved: "))
    self.payments_dict[year][3] += payment
    self.apartment_yearly_balance(year)
    ## Send a reciept to owner mail???

# **Class Building**
---


Each building is made of apartments. In order to "build" a building, you need to define the address, city and the number of apartments.
While creating a building, it is checked for being uniqe with __ new __ () method. 
Each building is given an ID number for further identification.

## **In: Building's Fees**

The fee of each apartment is calculated as the monthly fee required by the building, multiplied by the **Apartment.ratio** attribute. 

The mechanism to establish the fee of an apartment in a buiding can be calculated in one of two possible methods:
1. **Linear Fees (Default):** All apartments pay the same fees, regrdless of the area or unique features of an apartment. In this case, the ratio should be 1 divided by number of apartments in the building.
2. **Relative Fees:** the area of the appartment divided by the total area of all apartments (excl. parking, roof, warehouse etc..). Therfore bigger apartments will pay higher fees, and vice versa. 
In this case, the ratio should be the area of the apartment divided by the sum of areas of all apartments. 

In order to change the default ratio to a new one you should use the *Change_Ratio()* method. 

> **Tip:** The ratio attribute is year-specific. Therefore, ideally, you may change the ratio at the beginning of the year or right after setting the building yearly expenses.

## **Out: Building's Expenses**

There are some fixed kinds of expenses in a building. As a deafault we set in advence the most common expenditures sections, but the user has the ability to add additional expenditure sections as desired.

The pre-programed expenditure sections are: Cleaning, Elevator, Electricity, Gardening and Insurance. 

Each payment to a supplier should be documented with the update_yearly_costs(), and is related to a specific expenditure section.

## **Excess: Building's extra budget to unplanned events**

Each building can decide on it's own policy of how much money should be in the account at all times in order.
Excess can be set as a fixed amount (e.g. 2000) or a percent of the monthly expenses (e.g. 12%).
The excess is managed by the **Building.excess attribute** and changed by using the **excess_update()** function.

##**Instantiation  & Basic Functions**

In [ ]:
class Building():
  build_db = defaultdict(list)
  newid = itertools.count(1000)     # An iterator for unique property ID

  def __new__ (cls, city, street, house_num, app_num):
      """
      The __new__ method is similar to the __init__ method, but if both exist, __new__ method executes first.
      the __new__ method should check for duplicates in the dictionary prior to the instantiation phase
      """
      for item in Building.build_db.items():
         if item[1][0] == city and item[1][1] == street and item[1][2] == house_num and item[1][3] == app_num:
             print(f'This address already exists - property {item[0]}')
             return None
         else:
             continue
      return object.__new__(cls)

  def __init__ (self, city, street, house_num, app_num):
      global build_db, newid
      self.id = next(Building.newid)
      self.city = city
      self.street = street
      self.house_num = house_num
      self.app_num = app_num
      self.expenses_dict = defaultdict(list)
      self.yearly_expenses = 10000        #10000 is the default monthly fee for the building
      self.yearly_excess = 0           # 0 is the default monthly fee for the building
      self.f_yearly_excess = 0            # factor\percent of yearly excess is set to 0
      self.b_monthly_fee = int((self.yearly_expenses+self.yearly_excess)/12)        #1000 is the default monthly fee for the building
      Building.build_db[self.id] = [self.city, self.street, self.house_num, self.app_num, self.b_monthly_fee]
      self.apartments = [Apartment(self.id, i+1) for i in range(self.app_num)]
       

  def __str__ (self):
      try:
        building_string = ''
        header = '{:^8}|{:14}|{:15}|{:5}|{:8}|{:10}\n'.format('# Prop','City','Street','House','# Apart.', 'M.Fee(NIS)')
        line = '-'*len(header)+'\n'
        building_line = '{:^8}|{:14}|{:15}|{:5}|{:^8}|{:^10}\n'.format(self.id, self.city, self.street, self.house_num, self.app_num, self.b_monthly_fee)
        return (header+line+building_line)
      except TypeError:
        return 'No Occurance'
  
  def remove_property(id):
      Building.build_db.pop(id)
  
  def calc_yearly_expenses(self, year):
      self.yearly_expenses = 0
      for item in self.expenses_dict[year]:
        self.yearly_expenses += item[1]

  def set_yearly_expenses(self, year):
      self.yearly_expenses = 0

      # Entering pre-set expenditure costs
      cleaning = int(input("Enter Annual cleaning costs: "))
      elevator_main = int(input("Enter Annual elevator maintanance costs: "))
      electricity = int(input("Enter Annual electricity costs: "))
      gardening = int(input("Enter Annual gardening costs: "))
      insurance = int(input("Enter Annual insurance costs: "))
      self.expenses_dict[year] = [['Cleaning', cleaning, 0], ['Elevator', elevator_main, 0], ['Electricity', electricity, 0], ['Gardening', gardening, 0], ['Insurance', insurance, 0]]
      
      # Optional: Entering additionl costs
      user_answer1 = input('Is there any additional expenditure section? (Y/N)')
      while user_answer1 in 'Yy':
        exp_section_name = input("Enter expendituire section name: ")
        exp_section_cost = int(input(f"Enter {exp_section_name} annual cost: "))
        self.expenses_dict[year].append([exp_section_name, exp_section_cost, 0])
        user_answer1 = input('Are there any additional expenditure section?')
      
      # Calculating overall costs without deafault excess
      self.calc_yearly_expenses(year)
      
      # Calculating building excess
      user_answer2 = input(f'\nCurrent annual excess is set to {self.yearly_excess}.\nDo you want to change it? (Y/N) ')
      if user_answer2 in 'yY':
        self.excess_update(year)
      self.expenses_dict[year].append(['Excess', self.yearly_excess, 0]) 
      
      # Calculate expenses including excess and consequently the monthly fee of the building + printout
      self.calc_yearly_expenses(year)
      self.b_monthly_fee = round(self.yearly_expenses/12)
      Building.build_db[self.id][4] = self.b_monthly_fee
      self.push_apartments_fee(year)
      self.expenses_printout(year)
      # send an email????
  
  def update_yearly_costs(self, year):
      for i, section in enumerate(self.expenses_dict[year]):
        print(f'{i:2}.{section[0]}')
      cost_sec = input('Please select the section you want to update: ')
      cost = input('Please enter the expense (make sure you have an invoice!): ')
      self.expenses_dict[year][int(cost_sec)][2] += int(cost)
          
      
  def push_apartments_fee(self, year):
      for apart in self.apartments:
        apart.a_monthly_fee = round(self.b_monthly_fee * apart.ratio)
        apart.apartment_yearly_balance(year)
        apart.init_payments_dict(year)


  def excess_update(self, year):
      excess = input('Enter new excess in absolute NIS or percent of annual costs:')
      if '%' in excess:
          self.f_yearly_excess = int(excess[:excess.index('%')])
          self.yearly_excess = int((self.f_yearly_excess / 100) * self.yearly_expenses)
      else:
          self.yearly_excess = int(excess)

  def change_Ratio(self, year):
    print("------------------------------------------------------------------------------------------------------------------------")
    print("|ATTENTION!! You are going to change apartmens payment ratio. Make sure this is a beginning of a year or a new building!|")
    print("|Please prepare the list of ratios and enter it one-by-one.                                                             |")
    print("|make sure that sum of all ratios is 1. Otherwise an error will occur.                                                  |")
    print("------------------------------------------------------------------------------------------------------------------------\n")
    user_input = input("Do you want to proceed? (Y/N) ")
    while user_input in 'Yy':
      new_ratio_list = []
      for i, apart in enumerate(self.apartments, 1):
        new_ratio = input(f"Enter new ratio for apartment number {i}: ")
        new_ratio_list.append(new_ratio)
      s = sum(Decimal(i) for i in new_ratio_list)
      if s != 1.0:
        print('!!! Sum of ratios is different from 1. Please try again !!!')
        user_input = input("Do you want to proceed? (Y/N) ")
      else:
        for i in range(len(self.apartments)):
          self.apartments[i].ratio = float(new_ratio_list[i])
          self.apartments[i].a_monthly_fee = round(self.apartments[i].ratio * self.b_monthly_fee)
          self.apartments[i].payments_dict[year][0] = self.apartments[i].ratio
          self.apartments[i].payments_dict[year][1] = self.apartments[i].a_monthly_fee
        print('Ratios changed succesfully!')
        break

# -------------- BUILDING REPORTS ------------------ #
  
  def expenses_printout(self,year):
    #Report 2: A table of all expenses of a building in the given year
    my_str = ''
    header1 = f'Property {self.id}: {year} Expense Sheet\n'
    header2 = f'{self.city} {self.street} {self.house_num}\n'
    my_str = header1 + header2
    my_str = my_str + f'Annual expected Costs: {self.yearly_expenses} NIS\n'
    my_str = my_str + 'Exp. section | Estimated Costs | Actual Costs |  %  \n'
    my_str = my_str + '-'*50 + '\n'
    for item in self.expenses_dict[year]:
      my_str = my_str + f'{item[0]:13}|{item[1]:^17}|{item[2]:^14}|{(item[2]/item[1]):.1%}\n'
    return my_str 
  
  def print_all_buildings():
    #Report 3: A table of all the managed buildings
    build_string = ''
    header = '{:^6}|{:^12}|{:^15}|{:7}\n'.format('SN','City','Street','# house')
    line = '-'*len(header)+'\n'
    for build in Building.build_db.items():
        build_line = '{:^6}|{:12}|{:15}|{:^7}\n'.format(build[0], build[1][0], build[1][1], build[1][2])
        build_string += build_line
    return (header + line + build_string)
  
  def apartments_in_build(self, year):
    #Report 4: A table of all apartments in a given building in the given year including an up to date balance
    apartment_string = ''
    header = '{:9}|{:^6}|{:^8}|{:15}|{:^9}\n'.format('# Apart', 'Ratio', 'M. Fee', 'Owner Name', 'Balance')
    line = '-'*len(header)+'\n'
    for apart in self.apartments:
      apart.apartment_yearly_balance(year)
      apartment_line = '{:^9}|{:^6.1%}|{:^8}|{:15}|{:^9}\n'.format(apart.num, apart.payments_dict[year][0], apart.payments_dict[year][1], apart.owner_name, apart.payments_dict[year][2])
      apartment_string += apartment_line
    return (header+line+apartment_string)

  def apartment_owner_balance(owner, year):
    #Report 5: A table of all the apartments of a given owner and its current balance
    aob_string = ''
    header = '{:^9}|{:^29}|{:^8}|{:^8}\n'.format('Apart #','Address','Paid','Balance')
    line = '-'*len(header)+'\n'
    for house in Building.build_db.items():
      prop = eval(f'house'+str(house[0]))
      for apart in prop.apartments:
        if apart.owner == owner:
          owner_apart_line = f'{apart.num:^9}|{house[1][0]:12} {house[1][1]:12} {house[1][2]:3}|{apart.payments_dict[year][3]:^8}|{apart.payments_dict[year][2]:^8}\n'
          aob_string += owner_apart_line
          cur_name = apart.owner_name
        else:
          continue
    con = f'Owner Report {cur_name}: {year}\n'
    return (con+header+line+aob_string)

##**Demonstration**

In [ ]:
# Create 2 new buildings and print it
house1000 = Building('Petah Tiqwa', 'Ehad Haam', '15', 8)
house1001 = Building('Tel-Aviv', 'Dizingof', '32', 4)
print(house1000)
print(house1001)

 # Prop |City          |Street         |House|# Apart.|M.Fee(NIS)
------------------------------------------------------------------
  1000  |Petah Tiqwa   |Ehad Haam      |15   |   8    |   833    

 # Prop |City          |Street         |House|# Apart.|M.Fee(NIS)
------------------------------------------------------------------
  1001  |Tel-Aviv      |Dizingof       |32   |   4    |   833    



In [ ]:
# Set yearly expense for property 1000
house1000.set_yearly_expenses(2020)

Enter Annual cleaning costs: 2000
Enter Annual elevator maintanance costs: 5000
Enter Annual electricity costs: 2500
Enter Annual gardening costs: 3500
Enter Annual insurance costs: 4200
Is there any additional expenditure section? (Y/N)y
Enter expendituire section name: Solar
Enter Solar annual cost: 1500
Are there any additional expenditure section?n

Current annual excess is set to 0.
Do you want to change it? (Y/N) y
Enter new excess in absolute NIS or percent of annual costs:8%


In [ ]:
# Check the length & the content of the building database. each building is a tuple.
print('There are currently',len(Building.build_db.items()),'managed properties')
Building.build_db.items()

There are currently 2 managed properties


dict_items([(1000, ['Petah Tiqwa', 'Ehad Haam', '15', 8, 1683]), (1001, ['Tel-Aviv', 'Dizingof', '32', 4, 833])])

In [ ]:
# A Try to build an additional building in the same address fails
house1002 = Building('Tel-Aviv', 'Dizingof', '32', 4)
print('There are currently',len(Building.build_db.items()),'managed properties')

This address already exists - property 1001
There are currently 2 managed properties


In [ ]:
# attach owners to some apartment in building #1 (# prop 1000). 
house1000.apartments[0].add_owner('012345678')
house1000.apartments[1].add_owner('067983451')
house1000.apartments[2].add_owner('012345678')
house1001.apartments[3].add_owner('067983451')

print(house1000.apartments[0])

# Apart  |Ratio | M. Fee |Owner Name     | Balance 
----------------------------------------------------
 1000 / 1|12.5% |  210   |Carmit Assael  |  2520   



In [ ]:
house1000.update_yearly_costs(2020)

 0.Cleaning
 1.Elevator
 2.Electricity
 3.Gardening
 4.Insurance
 5.Solar
 6.Excess
Please select the section you want to update: 0
Please enter the expense (make sure you have an invoice!): 1200


##**Report 2: Property Expenses Sheet**

In [ ]:
print(house1000.expenses_printout(2020))

send_HTML_report(house1000.expenses_printout(2020),'pelanat1207@gmail.com','Anat','Expenses Sheet')

Property 1000: 2020 Expense Sheet
Petah Tiqwa Ehad Haam 15
Annual expected Costs: 20196 NIS
Exp. section | Estimated Costs | Actual Costs |  %  
--------------------------------------------------
Cleaning     |      2000       |     1200     |60.0%
Elevator     |      5000       |      0       |0.0%
Electricity  |      2500       |      0       |0.0%
Gardening    |      3500       |      0       |0.0%
Insurance    |      4200       |      0       |0.0%
Solar        |      1500       |      0       |0.0%
Excess       |      1496       |      0       |0.0%

Type your password and press enter: ··········


##**Report 3: Assets List**

In [ ]:
print(Building.print_all_buildings())
send_HTML_report(Building.print_all_buildings(),'pelanat1207@gmail.com','Anat','Assets List')

  SN  |    City    |    Street     |# house
--------------------------------------------
 1000 |Petah Tiqwa |Ehad Haam      |  15   
 1001 |Tel-Aviv    |Dizingof       |  32   

Type your password and press enter: ··········


##**Report 4: Apartments List per Building (inc. balance)**

In [ ]:
print(house1000.apartments_in_build(2020))

# Apart  |Ratio | M. Fee |Owner Name     | Balance 
----------------------------------------------------
 1000 / 1|12.5% |  210   |Carmit Assael  |  2520   
 1000 / 2|12.5% |  210   |Anat Peled     |  2520   
 1000 / 3|12.5% |  210   |Carmit Assael  |  2520   
 1000 / 4|12.5% |  210   |               |  2520   
 1000 / 5|12.5% |  210   |               |  2520   
 1000 / 6|12.5% |  210   |               |  2520   
 1000 / 7|12.5% |  210   |               |  2520   
 1000 / 8|12.5% |  210   |               |  2520   



In [ ]:
house1000.apartments[0].get_payment(2020)

Enter the sum of payment recieved: 128


In [ ]:
print(house1000.apartments_in_build(2020))
send_HTML_report(house1000.apartments_in_build(2020),'pelanat1207@gmail.com','Anat','Apartments List')

# Apart  |Ratio | M. Fee |Owner Name     | Balance 
----------------------------------------------------
 1000 / 1|12.5% |  210   |Carmit Assael  |  2392   
 1000 / 2|12.5% |  210   |Anat Peled     |  2520   
 1000 / 3|12.5% |  210   |Carmit Assael  |  2520   
 1000 / 4|12.5% |  210   |               |  2520   
 1000 / 5|12.5% |  210   |               |  2520   
 1000 / 6|12.5% |  210   |               |  2520   
 1000 / 7|12.5% |  210   |               |  2520   
 1000 / 8|12.5% |  210   |               |  2520   

Type your password and press enter: ··········


In [ ]:
print(house1000.apartments[0].payments_dict.items())

dict_items([(2020, [0.125, 210, 2392, 128])])


In [ ]:
house1001.change_Ratio(2020)
print(house1001.apartments_in_build(2020))

------------------------------------------------------------------------------------------------------------------------
|ATTENTION!! You are going to change apartmens payment ratio. Make sure this is a beginning of a year or a new building!|
|Please prepare the list of ratios and enter it one-by-one.                                                             |
|make sure that sum of all ratios is 1. Otherwise an error will occur.                                                  |
------------------------------------------------------------------------------------------------------------------------

Do you want to proceed? (Y/N) y
Enter new ratio for apartment number 1: 0.3
Enter new ratio for apartment number 2: 0.2
Enter new ratio for apartment number 3: 0.1
Enter new ratio for apartment number 4: 0.4
Ratios changed succesfully!
# Apart  |Ratio | M. Fee |Owner Name     | Balance 
----------------------------------------------------
 1001 / 1|30.0% |  250   |               |  3000  

##**Report 5: Owner balance sheet**

In [ ]:
print(Building.apartment_owner_balance('067983451', 2020))
send_HTML_report(Building.apartment_owner_balance('067983451', 2020),'pelanat1207@gmail.com','Anat','Owner balance sheet')

Owner Report Anat Peled: 2020
 Apart # |           Address           |  Paid  |Balance 
----------------------------------------------------------
 1000 / 2|Petah Tiqwa  Ehad Haam    15 |   0    |  2520  
 1001 / 4|Tel-Aviv     Dizingof     32 |   0    |  3996  

Type your password and press enter: ··········


In [ ]:
print(Building.apartment_owner_balance('012345678', 2020))
send_HTML_report(Building.apartment_owner_balance('012345678', 2020),'pelanat1207@gmail.com','Anat','Owner balance sheet')

Owner Report Carmit Assael: 2020
 Apart # |           Address           |  Paid  |Balance 
----------------------------------------------------------
 1000 / 1|Petah Tiqwa  Ehad Haam    15 |  128   |  2392  
 1000 / 3|Petah Tiqwa  Ehad Haam    15 |   0    |  2520  

Type your password and press enter: ··········


#**Summary**
---


This project is the first of five assignments required in Data Science Course of Naya College and submitted to Dror Geva.

The project demonstras the use of a multi-class application and its complexity. 

This application is setting the basis towards a more advanced house managment application, that can be further devoloped in mulitple directions, including, for example:
1. User Interface approach (instead of interactive inputs)
2. Graphical add-in for finance variables and reports
3. Pre-scheduled reports



*All rights reserved to Anat Peled, July 2020*